In [17]:
import torch
import ray
from ray import tune
from ray.air import session
from sklearn.model_selection import train_test_split
import yaml
import argparse
import shutil
from collections import namedtuple
import os
import datetime


# Custom Libraries
from utils.data_generator import DataGenerator
from utils.agent import TrainModel
from utils.helper import to_python_native, gen_experiment_name, set_seed, save_model_state
from utils.plotter import alignment_progress_over_iterations, plot_initial_final_accuracies
from utils.pennylane.model import Qkernel

In [18]:
# === Backend Configuration ===
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.xpu.is_available():
    device = torch.device("xpu")
else:
    device = torch.device("cpu")

set_seed(42)

In [19]:
data_generator = DataGenerator(
        dataset_name= 'wine',
        file_path=None,
    )

training_data, training_labels, testing_data, testing_labels = data_generator.generate_dataset()
training_data = torch.tensor(training_data.to_numpy(), dtype=torch.float32, requires_grad=True)
testing_data = torch.tensor(testing_data.to_numpy(), dtype=torch.float32, requires_grad=True)
training_labels = torch.tensor(training_labels.to_numpy(), dtype=torch.int)
testing_labels = torch.tensor(testing_labels.to_numpy(), dtype=torch.int)

In [20]:
print(training_data.size())
print(testing_data.size())

torch.Size([142, 13])
torch.Size([36, 13])


In [21]:
kernel = Qkernel(
        device='lightning.qubit',
        n_qubits=4,
        trainable=True,
        input_scaling=True,
        data_reuploading=True,
        ansatz='embedding_paper',
        ansatz_layers=5
    )

In [22]:
agent = TrainModel(
        kernel=kernel,
        training_data=training_data,
        training_labels=training_labels,
        testing_data=testing_data,
        testing_labels=testing_labels,
        optimizer='gd',
        lr= 0.1,
        mclr=0.01,
        cclr=0.01,
        epochs=100,
        train_method='ccka',
        target_accuracy=0.95,
        get_alignment_every=100,
        validate_every_epoch=1,
        base_path='./',
        lambda_kao=0.001,
        lambda_co=0.001,
        clusters=8,
        use_kmeans=True
    )

Epochs:  10


In [23]:
before_metrics = agent.evaluate_test(testing_data, testing_labels, 'before')
before_metrics

proto_labels: [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]
len(subcentroids): 24
{1: [Parameter containing:
tensor([ 1.4763, -1.9548, -0.1320, -1.3233,  0.0195,  1.1885,  0.5378, -1.6004,
         0.1656, -0.7038, -0.6823,  1.8922, -0.2004], requires_grad=True), Parameter containing:
tensor([ 0.8267, -1.9992,  0.4296, -1.0688, -1.0049, -0.0588, -0.2613, -1.0416,
        -0.9528, -1.5654,  0.1788,  0.2844,  0.5135], requires_grad=True), Parameter containing:
tensor([ 1.3662, -1.7571,  0.8862, -0.7692, -0.3415,  1.1266,  0.5915, -0.4149,
        -0.0768, -0.3545,  0.0575,  0.6128,  1.1949], requires_grad=True), Parameter containing:
tensor([ 1.8850, -1.7353, -0.0588, -2.2185, -1.2635,  1.2729,  1.0207, -1.3930,
         1.5262, -0.3632,  0.5619,  0.7307,  1.6145], requires_grad=True), Parameter containing:
tensor([ 0.5556, -0.5041,  1.5826,  0.0000, -0.3551,  0.5547, -0.0451, -0.4149,
        -0.3310, -1.5483, -0.2094,  1.3234, -0.8649], requires_grad=True), P

{'alignment': tensor(0.9346, grad_fn=<SqueezeBackward0>),
 'executions': None,
 'training_accuracy': 0.6690140845070423,
 'testing_accuracy': 0.6944444444444444,
 'f1_score': 0.7061294261294262,
 'alignment_arr': [[], []],
 'loss_arr': [],
 'validation_accuracy_arr': []}

In [24]:
 agent.fit_multiclass(training_data, training_labels)

Started Training
------------------------------------------------------------------
Epoch: 9 — Alignment per main centroid
Centroid 0 (label=0): Alignment = -0.3625839352607727
Centroid 1 (label=1): Alignment = -0.14122827351093292
Centroid 2 (label=2): Alignment = -0.03269040957093239
------------------------------------------------------------------


(Qkernel(),
 [Parameter containing:
  tensor([[1.0021, 0.9009, 1.0598, 1.6565],
          [1.0492, 0.3985, 1.1188, 1.5979],
          [0.5470, 0.9987, 0.6590, 1.2983],
          [1.1728, 1.7096, 1.5389, 1.0977],
          [0.1672, 1.5473, 1.1811, 0.9918]], requires_grad=True),
  Parameter containing:
  tensor([[ 2.5002,  1.9146, -1.1743,  3.6822],
          [-0.6125,  0.6750, -1.4409,  0.8102],
          [ 1.9916, -1.6664,  2.6247,  1.0600],
          [ 2.8034,  0.5125,  1.7722, -0.3050],
          [ 2.2808,  0.4373, -1.3812,  0.7541]], requires_grad=True),
  Parameter containing:
  tensor([[-1.5987, -0.1881, -1.4192,  1.7429],
          [-2.2155, -0.5051, -0.4418, -1.7258],
          [-0.1265, -3.3522,  3.1386, -2.2544],
          [ 2.0066,  0.3211, -1.1190,  2.0040],
          [ 0.4611,  2.3905,  0.3234, -0.9331]], requires_grad=True)],
 ParameterList(
     (0): Parameter containing: [torch.float32 of size 13]
     (1): Parameter containing: [torch.float32 of size 13]
     (2): Param

In [25]:
after_metrics = agent.evaluate_test(testing_data, testing_labels, 'after')
after_metrics

proto_labels: [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]
len(subcentroids): 24
{1: [Parameter containing:
tensor([ 1.5006, -2.0174, -0.1951, -1.4529,  0.0195,  1.1885,  0.5378, -1.6004,
         0.1656, -0.7038, -0.6823,  1.8922, -0.2004], requires_grad=True), Parameter containing:
tensor([ 0.7975, -2.0743,  0.3304, -1.0701, -1.0049, -0.0588, -0.2613, -1.0416,
        -0.9528, -1.5654,  0.1788,  0.2844,  0.5135], requires_grad=True), Parameter containing:
tensor([ 1.3649, -1.7673,  1.0042, -0.7004, -0.3415,  1.1266,  0.5915, -0.4149,
        -0.0768, -0.3545,  0.0575,  0.6128,  1.1949], requires_grad=True), Parameter containing:
tensor([ 1.8406, -1.7726, -0.2215, -2.1847, -1.2635,  1.2729,  1.0207, -1.3930,
         1.5262, -0.3632,  0.5619,  0.7307,  1.6145], requires_grad=True), Parameter containing:
tensor([ 0.5547, -0.5036,  1.5747,  0.0180, -0.3551,  0.5547, -0.0451, -0.4149,
        -0.3310, -1.5483, -0.2094,  1.3234, -0.8649], requires_grad=True), P

{'alignment': tensor(0.9647, grad_fn=<SqueezeBackward0>),
 'executions': 7200,
 'training_accuracy': 0.6197183098591549,
 'testing_accuracy': 0.6666666666666666,
 'f1_score': 0.6679012345679012,
 'alignment_arr': [[], []],
 'loss_arr': [],
 'validation_accuracy_arr': []}